In [1]:
using JLD2
using Statistics
using Revise
using Plots
using Laplacians

In [2]:
@load "spielmanIPM_all.jl.split02merge02.jld2"
dic1 = copy(dic)

Dict{Any,Any} with 49 entries:
  "hypre_solve"       => [0.24, 0.24, 0.22, 0.2, 0.2, 0.3, 0.21, 0.2, 0.21, 0.2…
  "icc2_tot"          => [6.37777, 5.6349, 5.95233, 6.46598, 8.00571, 5.48614, …
  "petsc_hypre_build" => [0.387031, 0.386556, 0.383702, 0.409836, 0.383895, 0.3…
  "lamg_its"          => [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0  …  …
  "ac_tot"            => [0.253683, 0.210542, 0.165944, 0.166443, 0.166224, 0.1…
  "icc2_solve"        => [6.18821, 5.4554, 5.76543, 6.27385, 7.8149, 5.28723, 5…
  "hypre_tot"         => [0.37, 0.37, 0.36, 0.35, 0.35, 0.46, 0.34, 0.35, 0.37,…
  "ac-s2m2_its"       => [7.0, 5.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, 4.0, 4.0  …  …
  "petsc_hypre_solve" => [1.41793, 0.614656, 0.384447, 0.711038, 1.47429, 0.310…
  "petsc_hypre_its"   => [17.0, 7.0, 5.0, 9.0, 19.0, 4.0, 5.0, 5.0, 5.0, 5.0  ……
  "icc_solve"         => [3.40171, 3.93851, 3.52589, 3.27953, 3.27106, 3.59101,…
  "icc2_its"          => [322.0, 287.0, 307.0, 323.0, 342.0, 284.0, 280.0, 288

In [3]:
import Laplacians.initDictCol!
dics = [Dict(),Dict(),Dict(),Dict(),Dict(),Dict()]

for key in keys(dic1)
    for i in 1:length(dics)
        initDictCol!(dics[i], key, typeof(dic1[key][1]))
        #@show dics[i]
    end
end


for i in 1:length(dic1["testName"])
    tn = dic1["testName"][i]
    curK = parse(Int64, (split(tn, ".")[2][2:end]))
    kInd = round(Int64, curK / 100)
    for key in keys(dic1)
        if key == "names"
            continue
        end
        push!(dics[kInd][key], dic1[key][i])
    end
end

for i in 1:length(dics)
    dics[i]["names"] = dic1["names"]
end

In [4]:
function nanErr!(d)
    for (key, value) in d
        if occursin("err", key)
            for i in 1:length(value)
                if isnan(value[i])
                    d[key][i] = Inf
                end
            end
        end
    end
end

nanErr! (generic function with 1 method)

In [5]:
nanErr!.(dics);

In [6]:
using Printf

function sd(d,x)
    #scale = exp10(ceil(log10(abs(x)))-d)
    #return round(x/scale)*scale
    return round(x, sigdigits=3)
end        
ff(s) = (@sprintf "%.4g" sd(3,s))
nf(s) = (@sprintf "%d" sd(3,s))
ffK(s) = (@sprintf "%.4g" sd(3,s)/1000)
nfK(s) =  round(Int64, sd(3,s)/1000)
ffe8(s) = (@sprintf "%.3g" sd(3,s)*1e8)
ffe6(s) = (@sprintf "%.3g" sd(3,s)*1e6)
ffmicro(s) = (@sprintf "%.4g" sd(3,s)*1e6)
ffe3(s) = (@sprintf "%.3g" sd(3,s)*1e3)

ffe3 (generic function with 1 method)

In [7]:
function labelTvn(tvn, err)
    if err <= 1e-8
        return ffe6(tvn)
    elseif err < 1e-4
        return string(ffe6(tvn), "\\tnote{*}")
    elseif err < 1
        return string(ffe6(tvn), "\\tnote{**}")
    else
        return "Inf"
    end
end

function printMedian(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = median(d[string(solver, "_tot")] ./ d["ne"] )
        err = median(d[string(solver, "_err")])
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

function print75(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = quantile(d[string(solver, "_tot")] ./ d["ne"], 0.75)
        err = quantile(d[string(solver, "_err")], 0.75)
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

function printMax(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = maximum(d[string(solver, "_tot")] ./ d["ne"])
        err = maximum(d[string(solver, "_err")])
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

printMax (generic function with 1 method)

In [8]:
function printSolverCell(d, solver) 
    return string(printMedian(d, solver), "&", print75(d, solver), "&", printMax(d, solver))
end

function printOneLine(d, solvers, i)
    allCells = [printSolverCell(d, solver) for solver in solvers]
    println(string(ffK(d["ne"][1]),"&", length(d["testName"]), "&", join(allCells, "&"), "\\\\"))
end

function printAllLines(dics, solvers)
    for i in 1:length(dics)
        printOneLine(dics[i], solvers, i)
    end
end

printAllLines (generic function with 1 method)

In [9]:
solvers=["ac", "ac-s2m2", "cmg2"]
printAllLines(dics, solvers)

1030&10&0.162&0.172&0.247&0.28&0.287&0.313&1.96&1.98&2.05\\
8100&10&0.194&0.195&0.223&0.296&0.305&0.369&2&2.07&2.22\\
2.72e+04&10&0.229&0.25&0.286&0.346&0.354&0.389&2.35&2.37&2.4\\
6.44e+04&10&0.247&0.268&0.291&0.357&0.367&0.396&2.48&2.53&2.65\\
1.26e+05&10&0.264&0.279&0.362&0.422&0.428&0.428&2.55&2.55&3.07\\
2.17e+05&11&0.418&0.437&0.735&0.502&0.51&0.809&2.64&2.72&3.29\\


In [10]:
solvers = ["hypre", "petsc_hypre", "icc2"]
printAllLines(dics, solvers)

1030&10&0.346&0.361&0.449&0.76&1.06&1.81\tnote{*}&5.65&6.13&7.81\\
8100&10&0.454&0.48&1.22&1.22&1.85&3.32\tnote{*}&12&12.1&12.3\\
2.72e+04&10&0.533&0.551&0.621&1.84&2.47&2.61&18.4&20.3&22.8\tnote{*}\\
6.44e+04&10&0.583&0.62&0.725&1.68&2.05&3.27&21\tnote{*}&22.3\tnote{*}&23.7\tnote{*}\\
1.26e+05&10&0.633&0.665&0.768&2.38&2.6&3.24&14.7&20.6&22.4\tnote{*}\\
2.17e+05&11&0.643&0.693&0.811&2.31&2.82&3.98&22.7\tnote{*}&22.7\tnote{*}&23\tnote{*}\\
